In [ ]:
# nn_spoof_detection.py
# Neural Network-Based GPS Spoofing Detection (classification only) using PyTorch
# Adapted from Jullian et al., "Deep Learning Detection of GPS Spoofing" (2022)

import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# ==== CONFIG ====
CSV_PATH = '/content/drive/MyDrive/thesis/combined_data_Auto_pilot.csv'
BATCH_SIZE = 32
EPOCHS = 15
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEQ_LEN = 20           # number of timesteps per sample
FEATURES = 10          # accel_x/y/z, gyro_x/y/z, speed, steering_angle, throttle, brake
USE_LSTM = True        # set False to use a pure MLP instead of LSTM

# ==== Dataset ====
class GPSSpoofSequenceDataset(Dataset):
    """
    Constructs rolling windows of sensor data for sequence-based spoofing detection.
    Label for each window is the state ('spoofed' or not) at the last timestep.
    """
    def __init__(self, csv_path, seq_len):
        df = pd.read_csv(csv_path)
        self.seq_len = seq_len
        # extract sensor columns
        self.sensors = df[[
            'accel_x', 'accel_y', 'accel_z',
            'gyro_x', 'gyro_y', 'gyro_z',
            'speed', 'steering_angle', 'throttle', 'brake'
        ]].values.astype('float32')
        # binary labels: 1 for spoofed, 0 otherwise
        self.labels = (df['label'] == 'spoofed').astype('float32').values

    def __len__(self):
        # number of full sequences
        return len(self.sensors) - self.seq_len + 1

    def __getitem__(self, idx):
        seq = self.sensors[idx:idx + self.seq_len]
        label = self.labels[idx + self.seq_len - 1]
        return torch.from_numpy(seq), torch.tensor(label)

# ==== Models ====
class MLPDetector(nn.Module):
    def __init__(self, seq_len, feature_dim):
        super().__init__()
        input_dim = seq_len * feature_dim
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: (batch, seq_len, features)
        x_flat = x.view(x.size(0), -1)
        return self.net(x_flat).squeeze(1)

class LSTMDetector(nn.Module):
    def __init__(self, feature_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(feature_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=0.3)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: (batch, seq_len, features)
        _, (h_n, _) = self.lstm(x)
        # take last layer's hidden state
        h_last = h_n[-1]
        return self.classifier(h_last).squeeze(1)

# ==== Prepare DataLoader ====
dataset = GPSSpoofSequenceDataset(CSV_PATH, SEQ_LEN)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# ==== Initialize Model ====
if USE_LSTM:
    model = LSTMDetector(feature_dim=FEATURES).to(DEVICE)
else:
    model = MLPDetector(seq_len=SEQ_LEN, feature_dim=FEATURES).to(DEVICE)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print("\n🚀 Training spoofing detection network...")
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    for seq, label in train_loader:
        seq, label = seq.to(DEVICE), label.to(DEVICE)

        pred = model(seq)
        loss = criterion(pred, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f}")

# ==== Save model ====
torch.save(model.state_dict(), 'nn_spoof_detection_model.pth')
print("✅ Model saved as nn_spoof_detection_model.pth")
